In [8]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps, operator_average
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin, get_h1_spin, get_g_spin, get_t
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt, generate_Hamiltonian_with_occupation_number
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from pytenet.operation_thc import get_num_op_mpo


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/NH3/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/NH3/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 36)
(36, 36)
rl errV: 6.421105521729929e-13
abs errV: 4.286591936668418e-12
errt: 1.0574710399920436e-13
errh: 1.8088404989173088e-14
errht: 9.006422291220954e-14


In [3]:
# H_correct = get_molecular_Hamiltonian_as_sparse_matrix(get_h1_spin(h1), get_g_spin(eri))
# H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H_correct_NH3.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

In [4]:
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [5]:
e_ground = -67.46923609481743
e_1st_ex = -67.00839085980371

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [6]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 62, 100, 154, 100, 62, 16, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [9]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state.

In [11]:
#temp1 = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])
initial = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0])

initial = generate_single_state(8, [3, 3, 3, 3, 1, 1, 0, 0])

#hartree_state = add_mps(add_mps(add_mps(temp1, temp2), temp3),temp4)
# hartree_state = add_mps(temp1, temp2)
# hartree_state.orthonormalize('left')
# hartree_state.orthonormalize('right')

generate Krylov space:

In [12]:
N_Krylov_1 = 35
psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 150
trunc_tol = 1e-10
foldername_1 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex"
generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

[1, 4, 10, 17, 23, 20, 11, 4, 1]
2
[1, 4, 15, 38, 52, 41, 15, 4, 1]
3
[1, 4, 16, 50, 77, 53, 16, 4, 1]
4
[1, 4, 16, 54, 90, 55, 16, 4, 1]
5
[1, 4, 16, 56, 94, 56, 16, 4, 1]
6
[1, 4, 16, 56, 94, 56, 16, 4, 1]
7
[1, 4, 16, 56, 94, 56, 16, 4, 1]
8
[1, 4, 16, 56, 96, 56, 16, 4, 1]
9
[1, 4, 16, 56, 96, 56, 16, 4, 1]
10
[1, 4, 16, 56, 95, 56, 16, 4, 1]
11
[1, 4, 16, 56, 96, 56, 16, 4, 1]
12
[1, 4, 16, 56, 96, 56, 16, 4, 1]
13
[1, 4, 16, 56, 96, 56, 16, 4, 1]
14
[1, 4, 16, 56, 96, 56, 16, 4, 1]
15
[1, 4, 16, 56, 95, 56, 16, 4, 1]
16
[1, 4, 16, 56, 96, 56, 16, 4, 1]
17
[1, 4, 16, 56, 96, 56, 16, 4, 1]
18
[1, 4, 16, 56, 96, 56, 16, 4, 1]
19
[1, 4, 16, 56, 96, 56, 16, 4, 1]
20
[1, 4, 16, 56, 95, 56, 16, 4, 1]
21
[1, 4, 16, 56, 96, 56, 16, 4, 1]
22
[1, 4, 16, 56, 96, 56, 16, 4, 1]
23
[1, 4, 16, 56, 96, 56, 16, 4, 1]
24
[1, 4, 16, 56, 96, 56, 16, 4, 1]
25
[1, 4, 16, 56, 96, 56, 16, 4, 1]
26
[1, 4, 16, 56, 96, 56, 16, 4, 1]
27
[1, 4, 16, 56, 96, 56, 16, 4, 1]
28
[1, 4, 16, 56, 96, 56, 16, 4, 1]
29


In [13]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)
H_reduced_1 = remain_only_tridiagonal_elements(H_reduced_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:213: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [14]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_1st_ex, mpo_ref)

(0.4341310427576275+0j)
(0.03046646361077876+0j)
(0.008662569737396097+0j)
(0.0065919828738145725+0j)


(0.005344603796075376+0j)
(0.00481023012167725+0j)
(0.004696966248431522+0j)


In [15]:
# spin = 0
# for i in range (L):
#     if i%2 == 0:
#         spin_local = 0.5* operator_average(v_ritz_1, get_num_op_mpo(i, L))
#     if i%2 == 1:
#         spin_local = -0.5* operator_average(v_ritz_1, get_num_op_mpo(i, L))
#     spin += spin_local 
# print(spin)

Restart:

In [16]:
N_Krylov_2 = 35
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 150
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_1"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)


[1, 4, 16, 56, 96, 56, 16, 4, 1]
2
[1, 4, 16, 56, 96, 56, 16, 4, 1]
3
[1, 4, 16, 56, 96, 56, 16, 4, 1]
4
[1, 4, 16, 56, 96, 56, 16, 4, 1]
5
[1, 4, 16, 56, 96, 56, 16, 4, 1]
6
[1, 4, 16, 56, 96, 56, 16, 4, 1]
7
[1, 4, 16, 56, 96, 56, 16, 4, 1]
8
[1, 4, 16, 56, 96, 56, 16, 4, 1]
9
[1, 4, 16, 56, 96, 56, 16, 4, 1]
10
[1, 4, 16, 56, 96, 56, 16, 4, 1]
11
[1, 4, 16, 56, 96, 56, 16, 4, 1]
12
[1, 4, 16, 56, 96, 56, 16, 4, 1]
13
[1, 4, 16, 56, 96, 56, 16, 4, 1]
14
[1, 4, 16, 56, 96, 56, 16, 4, 1]
15
[1, 4, 16, 56, 96, 56, 16, 4, 1]
16
[1, 4, 16, 56, 96, 56, 16, 4, 1]
17
[1, 4, 16, 56, 96, 56, 16, 4, 1]
18
[1, 4, 16, 56, 96, 56, 16, 4, 1]
19
[1, 4, 16, 56, 96, 56, 16, 4, 1]
20
[1, 4, 16, 56, 96, 56, 16, 4, 1]
21
[1, 4, 16, 56, 96, 56, 16, 4, 1]
22
[1, 4, 16, 56, 96, 56, 16, 4, 1]
23
[1, 4, 16, 56, 96, 56, 16, 4, 1]
24
[1, 4, 16, 56, 96, 56, 16, 4, 1]
25
[1, 4, 16, 56, 96, 56, 16, 4, 1]
26
[1, 4, 16, 56, 96, 56, 16, 4, 1]
27
[1, 4, 16, 56, 96, 56, 16, 4, 1]
28
[1, 4, 16, 56, 96, 56, 16, 4, 1]
29


In [17]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

In [18]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_1st_ex, mpo_ref)

(0.004637715597212377+0j)
(0.004396392857358933+0j)
(0.0035241255989433284+0j)
(0.0024277110478720942+0j)
(0.0010732931755228492+0j)
(0.0002379127858347374+0j)


(9.104645809543399e-05+0j)


In [24]:
# filename = foldername_2 + f"/Krylov_vec{10}.pkl"
# with open(filename, 'rb') as file:
#     vec_test = pickle.load(file)

# spin = 0
# for i in range (L):
#     if i%2 == 0:
#         spin_local = 0.5* operator_average(vec_test, get_num_op_mpo(i, L))
#     if i%2 == 1:
#         spin_local = -0.5* operator_average(vec_test, get_num_op_mpo(i, L))
#     spin += spin_local 
# print(spin)
# # the spin got errors after restart. Or, because the spin for v_ritz_1 has already been not strictly 1?

(0.9009664322028655+0j)


Restart:

In [19]:
N_Krylov_3 = 20
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 150
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_2"
generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)


[1, 4, 16, 56, 96, 56, 16, 4, 1]
2
[1, 4, 16, 56, 96, 56, 16, 4, 1]
3
[1, 4, 16, 56, 96, 56, 16, 4, 1]
4
[1, 4, 16, 56, 96, 56, 16, 4, 1]
5
[1, 4, 16, 56, 96, 56, 16, 4, 1]
6
[1, 4, 16, 56, 96, 56, 16, 4, 1]
7
[1, 4, 16, 56, 96, 56, 16, 4, 1]
8
[1, 4, 16, 56, 96, 56, 16, 4, 1]
9
[1, 4, 16, 56, 96, 56, 16, 4, 1]
10
[1, 4, 16, 56, 96, 56, 16, 4, 1]
11
[1, 4, 16, 56, 96, 56, 16, 4, 1]
12
[1, 4, 16, 56, 96, 56, 16, 4, 1]
13
[1, 4, 16, 56, 96, 56, 16, 4, 1]
14
[1, 4, 16, 56, 96, 56, 16, 4, 1]
15
[1, 4, 16, 56, 96, 56, 16, 4, 1]
16
[1, 4, 16, 56, 96, 56, 16, 4, 1]
17
[1, 4, 16, 56, 96, 56, 16, 4, 1]
18
[1, 4, 16, 56, 96, 56, 16, 4, 1]
19
[1, 4, 16, 56, 96, 56, 16, 4, 1]


In [20]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)
H_reduced_3 = remain_only_tridiagonal_elements(H_reduced_3)

In [21]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_1st_ex, mpo_ref)

(4.6058410660521076e-05+0j)
(1.146729286460868e-05+0j)
(3.8110286766368517e-06+0j)
(2.1414367097349896e-06+0j)


In [22]:
np.linalg.eigh(H_reduced_3)

EighResult(eigenvalues=array([-67.00838872, -66.81404485, -66.49069539, -65.79592652,
       -64.70254947, -63.6343909 , -62.39644198, -55.74557392,
       -51.85601768, -50.92426713, -49.36670301, -47.90790944,
       -47.04221742, -33.81077399, -33.1211225 , -31.96608447,
       -30.83264572, -30.0521206 , -29.20614183, -28.67710277]), eigenvectors=array([[ 9.99836788e-01, -1.73544627e-02, -4.32312672e-03,
         2.14398781e-03,  9.83001540e-04,  5.21292830e-04,
         1.84970967e-04,  3.48912774e-06,  3.88288322e-04,
         4.34991019e-04, -3.53484463e-04,  3.22681922e-04,
         2.53084458e-04,  2.39205780e-05,  4.97171557e-05,
        -7.07040985e-05,  8.10381341e-05, -6.32989458e-05,
        -3.12932097e-05, -9.14998211e-05],
       [-5.56169184e-03, -2.10928282e-01, -1.40006264e-01,
         1.62633570e-01,  1.41813638e-01,  1.10044065e-01,
         5.33741534e-02,  2.45873602e-03,  3.68115324e-01,
         4.37750608e-01, -3.90175272e-01,  3.85627857e-01,
         3.161